In [1]:
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
## Load API Keys for OpenAI (or LLM of choice) and Pinecone

OPENAI_API_KEY = "" ## Insert your OpenAI API key here or store it in an environment variable os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = "" ## Insert your OpenAI API key here or store it in an environment variable os.getenv('OPENAI_API_KEY')

## Model Selection

Select a model to test

In [5]:
## OpenAI GPT-4 Model
model_openai_gpt4o = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model='gpt-4o'
)

We will now link to the PDF file of the Thyroid Nodule Guidelines we are using for this project

## Load PDF

Load in the Thyroid Nodule Guidelines. Using Langchain, we can load the PDF and split into digestible documents.

In [8]:
from langchain_community.document_loaders import PyPDFLoader

## The actual guidelines are from 2015 but the paper was published in 2016, hence the discrepancy in filename

loader = PyPDFLoader('thyroid-nodule-guidelines-2016.pdf')
pages = loader.load_and_split()

In [9]:
## See how many documents the PDF was split into.
len(pages)

253

## Set up Pinecone Vector Store

Only run this code once as there is no need to duplicate creation of the vector store.

In [6]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings


os.environ['PINECONE_API_KEY'] =  PINECONE_API_KEY

index_name = "thyroid-rag-index"
embeddings = OpenAIEmbeddings()


## Use the below code to initialize a Pinecone vector store from the documents

# pinecone = vectorstore_from_docs = PineconeVectorStore.from_documents(
#     pages,
#     index_name=index_name,
#     embedding=embeddings
# )

In [8]:
## Use this code to retrieve the Pinecone vector store if you have already made one

pinecone = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

pinecone

## Develop the prompts

In [9]:

intro_template = """

I am an endocrine surgeon. You are an assistant that helps me find the best management for my patient using the uploaded guidelines.

Please read the guidelines and provide me the best management for my patient.

Please answer using the following format:

I believe that our patient has [diagnosis]. Given their [indication], I believe that they would benefit from [treatment].
In addition, they will require [additional treatments, pre-operative workup, follow-up, surveillance, etc].

Examples of diagnoses may be [benign thyroid nodule, papillary thyroid cancer]
Examples of indications may be [symptoms, nodule size, biopsy results etc]
Examples of treatments may be [total thyroidectomy, right thyroid lobectomy etc]. Always mention laterality when recommending an operation.
Examples of additional treatments may be [radioactive iodine ablation, surveillance ultraounds, measurement of thyroglobulin etc]

Do not waffle and be confident in your answers. Please also be concise.

Now I will give you some examples of your previous responses with feedback to help refine answers.

Example 1: 
Assistant: I believe that our patient has a multinodular goiter. Given their history of nodules increasing in size and tracheal deviation, 
I believe that they would benefit from a total thyroidectomy. This approach is recommended in patients with bilateral nodular disease who may
prefer to undergo one definitive surgery.

In addition, they will require postoperative surveillance, which may include measurement of thyroglobulin to monitor for any residual or
recurrent disease, and follow-up with periodic neck ultrasound. If the patient is found to have thyroid cancer postoperatively,
further treatment options, such as radioactive iodine ablation, may need to be considered based on the final histopathologic findings
and staging. Regular follow-up to monitor for recurrence or the development of symptoms, as well as surveillance ultrasounds and thyroid
function tests to ensure euthyroid status, will also be important.

Feedback: I liked that the GPT picked up on the tracheal deviation and that it correctly factored into the decision making. However, The GPT response
didn't indicate whether this was a toxic versus nontoxic multinodular goiter. An important part of the clinical decision making is understanding
whether the nodule is functional, so this was a glaring omission in the diagnosis. The GPT also did not suggest CT imaging to evaluate the extent of
the goiter. Preoperative work up should include evaluating the anatomy, sometimes the ultrasound is sufficient but for large goiters especially
those with compressive symptoms and tracheal deviation you want to get a CT scan to evaluate the extent of the goiter (both sub-sternal extent and
also its posterior extent). You should suggest additional imaging if the case suggests possible substernal extension.

Example 2: 
Assistant: I believe that our patient has recurrent papillary thyroid cancer. Given their raised thyroglobulin levels from 0.1 to 1.1,
confirmed papillary thyroid cancer in at least one lymph node by fine-needle aspiration, ultrasound findings indicating disease progression,
and a history of thyroidectomy with postoperative radioactive iodine (RAI) treatment, I believe that they would benefit from a lymph node
dissection to manage the persistent/recurrent disease. 

In addition, they will require close surveillance with neck ultrasounds and measurements of thyroglobulin levels to monitor for further disease
activity or response to treatment. The potential use of repeat PET-CT and/or other imaging modalities should be considered, based on the dynamics
of thyroglobulin level changes and clinical judgment. Repeat RAI therapy may also be discussed depending on the extent of the recurrent disease
and previous treatment response. The patient's risk stratification should be reassessed in a multidisciplinary tumor board setting to inform
the management plan, which may include additional localized treatments or systemic therapy. Management of TSH levels according to current
guidelines for persistent disease and monitoring for potential complications from treatment will also be important.

Feedback: GPT recognized the recurrent cancer and appropriately recommended surgery. However, GPT did not identify the correct operation to perform.
"Lymph node dissection" is a generic term that doesn't give us any information about what the actual operation is to be performed.
The correct answer is "Right lateral neck dissection" - this gives us the laterality and also the type of lymph node excision we will do.
Lateral neck dissection is a specific term that denotes exactly what neck compartments are going to be explored and emptied of all their
lymph node pockets. Please be specific when suggesting operations.

The GPT did not review the risks of the procedure. Especially as this is considered a reoperation because we will have to go back into a
previously operated field, the risks are significantly higher and need to be reviewed with the patient.

Example 3:
Assistant: I believe that our patient has thyroid nodules suspicious for papillary thyroid carcinoma. Given their Bethesda Category V fine-needle
aspiration result and sonographic features of the nodule being solid, hypoechoic with microcalcifications, and classified as ATA high risk,
I believe that they would benefit from a total thyroidectomy. This recommendation aligns with guidelines for suspicious nodules of significant
size with high-risk features.

In addition, they will require postoperative surveillance, which may include measurement of thyroglobulin to monitor for any residual or recurrent disease,
and periodic neck ultrasound. If papillary thyroid carcinoma is confirmed by the final pathology, additional treatments such as radioactive iodine ablation
may need to be considered based on the staging and risk stratification. Regular follow-up to ensure euthyroid status and to monitor for recurrence will also
be important.

Feedback: You chose the wrong operation (total thyroidectomy) because you saw the term ATA high risk classification. When risk stratification is mentioned it is important to determine whether it is regarding the imaging risk stratification versus histopathologic risk stratification. If “ATA risk” stratification is reported under “Ultrasound” or “imaging” then it should assume thats in reference to the imaging risk stratification system. If it the “ATA risk” is listed under “pathology” “histology” “histopathology” “biopsy” etc then it should be assumed that is referencing the histopathology risk stratification system.
For this case, ATA High Risk was referring to imaging, so it would not automatically warrant a total thyroidectomy and a right thyroid lobectomy would be
the correct answer

Example 4:
If you recommend a total thyroidectomy, you should also recommend thyroid hormone supplementation as part of the follow-up plan.

Example 5:
You do not need to monitor thyroglobulin for a patient with thyroid lobectomy. Thyroglobulin is a marker of thyroid tissue and only if the thyroid is removed
then there should be no thyroglobulin to monitor. You also don't do radioactive iodine for lobectomy. These are only useful if there is no thyroid tissue left.

Example 6:
Please be specific when suggesting an operation. It is not enough to say resection, you have to specify the type of resection (e.g. thyroid lobectomy vs. total thyroidectomy)

Example 7:
Clinical vignette included a patient with a large nodule with subclinical hyperthyroidism. In its diagnosis, the GPT's assessment only mentioned "benign thyroid nodule" without mention of the hyperthyroidism which is very important.

Example 8:
Clinical vignette included a patient with a benign nodule but reported symptoms of dysphagia and voice change. The GPT recommended surveillance instead of surgery. For patients with compressive symptoms secondary to a nodule, surgery should be offered.

Example 9:
A clinical vignette included FNA results demonstrating PTC in both a thyroid nodule and lymph nodes. The GPT's diagnosis was just "Papillary thyroid cancer" but it should have mentioned that this is "metastatic papillary thyroid cancer".

Please use the above examples to refine your responses.



Context: {context}

Clinical Scenario: {question}

"""


## Develop the chain

In [10]:

def run_chain(scenario, model_to_use):

    intro_prompt = ChatPromptTemplate.from_template(intro_template)

    model = model_to_use

    parser = StrOutputParser()

    chain = (
        {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
        | intro_prompt
        | model
        | parser
    )

    return chain.invoke(scenario)

## Iterate through chain multiple times

We will use five for example

In [11]:
## iterate through the chain n times

def iterate_chain(scenario, model_to_use, n):

    responses = []

    responses.append('There are ' + str(n) + ' experts responses\n')
    
    for i in range(n):
        response = run_chain(scenario, model_to_use)
        # print(response)
        responses.append(response)
    
    return responses

## Create a second chain to make a composite response

In [12]:
composite_template = """

I have made an endocrine surgery assistant that helps me find the best management for my patient using the uploaded guidelines. 

For each patient scenario, I ran that assistant multiple times to simulate multiple experts.

I will give you all of the answers and I want you to synthesize the most common components of each answer into a single answer.

When there is uncertainty, choose the most frequently appearing option.

I am doing this to improve accuracy and reduce hallucinations.


Please output the final answer using the following format:

I believe that our patient has [diagnosis]. Given their [indication], I believe that they would benefit from [treatment].
In addition, they will require [additional treatments, pre-operative workup, follow-up, surveillance, etc].

Examples of diagnoses may be [benign thyroid nodule, papillary thyroid cancer]
Examples of indications may be [symptoms, nodule size, biopsy results etc]
Examples of treatments may be [total thyroidectomy, right thyroid lobectomy etc]. Always mention laterality when recommending an operation.
Examples of additional treatments may be [radioactive iodine ablation, surveillance ultraounds, measurement of thyroglobulin etc]


Do not waffle and be confident in your answers.


Responses: {question}

"""


In [13]:
def composite_response(responses, model_to_use):

    ## turn responses into a string separated by a newline

    responses_string = '\n'.join(responses)

    composite_prompt = ChatPromptTemplate.from_template(composite_template)

    model = model_to_use

    parser = StrOutputParser()

    chain = (
        {"question": RunnablePassthrough()}
        | composite_prompt
        | model
        | parser
    )

    return chain.invoke(responses_string)
    
 

In [17]:
def run_endo_gpt(prompt, model, n):
    answer = composite_response(iterate_chain(prompt, model, n), model)
    print(answer)
    return answer

In [22]:
import pandas as pd

df = pd.read_csv('test_cases.csv')

df

,id,case,attending_ap
0,4_30_CM_MG,51 year old female who was discovered to have ...,I believe that our patient has a thyroid nodul...
1,4_30_JL_KF,66 year old female who was discovered to have ...,I believe that our patient has a non-toxic mul...
2,4_30_CM_KS,38 year old female who was discovered to have ...,I believe that our patient has a thyroid nodul...
3,4_30_JL_AB,She is a 53 year old female who was discovered...,I believe that our patient has an enlarging t...
4,4_30_CM_BH,89 year old female who was discovered to have ...,I believe that our patient has a multinodular ...
5,4_30_JL_JF,66 year old female who was discovered to have ...,I believe that our patient has a non-toxic mul...
6,4_30_JL_JA,65 year old male who was discovered to have th...,I believe that our patient has papillary thyro...
7,4_30_JL_EK,46 year old female who was discovered to have ...,I believe that our patient has a toxic multino...
8,4_30_JL_NC,26 year old male who was discovered to have th...,I believe that our patient has papillary thyro...
9,4_30_CM_SA,62 year old female who was discovered to have ...,I believe that our patient has an indeterminat...


In [23]:
df['endogpt_ap'] = df.apply(
    lambda x:
    run_endo_gpt(x['case'], model_openai_gpt4o, 5),
    axis=1
)

I believe that our patient has a multinodular goiter with compressive symptoms. Given their history of progressive enlargement of the right thyroid lobe and symptoms of dysphagia and neck fullness, I believe that they would benefit from a right thyroid lobectomy.

In addition, they will require preoperative imaging, such as a CT scan, to evaluate the extent of the goiter, particularly to assess any potential retrosternal extension. Postoperatively, the patient will need thyroid hormone supplementation to maintain euthyroid status. Regular follow-up with periodic neck ultrasounds and thyroid function tests will be important to monitor for recurrence or other complications.
I believe that our patient has multiple thyroid nodules, including at least one suspicious for malignancy (TIRADS 4). Given their history of radiation exposure, non-diagnostic FNA results, and symptoms of pooling saliva, I believe that they would benefit from a right thyroid lobectomy to achieve a definitive diagnosis

In [24]:
df

,id,case,attending_ap,endogpt_ap
0,4_30_CM_MG,51 year old female who was discovered to have ...,I believe that our patient has a thyroid nodul...,I believe that our patient has a multinodular ...
1,4_30_JL_KF,66 year old female who was discovered to have ...,I believe that our patient has a non-toxic mul...,I believe that our patient has multiple thyroi...
2,4_30_CM_KS,38 year old female who was discovered to have ...,I believe that our patient has a thyroid nodul...,I believe that our patient has thyroid nodules...
3,4_30_JL_AB,She is a 53 year old female who was discovered...,I believe that our patient has an enlarging t...,I believe that our patient has a benign thyroi...
4,4_30_CM_BH,89 year old female who was discovered to have ...,I believe that our patient has a multinodular ...,I believe that our patient has benign multinod...
5,4_30_JL_JF,66 year old female who was discovered to have ...,I believe that our patient has a non-toxic mul...,I believe that our patient has a multinodular ...
6,4_30_JL_JA,65 year old male who was discovered to have th...,I believe that our patient has papillary thyro...,I believe that our patient has metastatic papi...
7,4_30_JL_EK,46 year old female who was discovered to have ...,I believe that our patient has a toxic multino...,I believe that our patient has papillary thyro...
8,4_30_JL_NC,26 year old male who was discovered to have th...,I believe that our patient has papillary thyro...,I believe that our patient has papillary thyro...
9,4_30_CM_SA,62 year old female who was discovered to have ...,I believe that our patient has an indeterminat...,I believe that our patient has a benign thyroi...


In [25]:
df.to_csv('results.csv')

## If you want to see our results, see 'results_in_paper.csv'